# MNIST experiment prototype

In [ ]:
import torch
import numpy as np

In [ ]:
import json

from pkg_resources import resource_stream

with resource_stream("cplxpaper.mnist", "template.json") as fin:
    options = json.load(fin)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from cplxpaper.auto import auto

In [ ]:
options["device"] = "cuda:1"

In [ ]:
options["datasets"] = {
    'mnist-train': {
        'cls': "<class 'cplxpaper.mnist.dataset.FashionMNIST_Train'>", 'root': './data'
    },
    'mnist-test': {
        'cls': "<class 'cplxpaper.mnist.dataset.FashionMNIST_Test'>", 'root': './data'
    }
}

In [ ]:
options["features"] = {
#     'cls': "<class 'cplxpaper.auto.feeds.FeedFourierFeatures'>",
#     'signal_ndim': 2,
#     'cplx': True,
#     'shift': True,
    'cls': "<class 'cplxpaper.auto.feeds.FeedRawFeatures'>"
}

In [ ]:
options["model"]["cls"] = "<class 'cplxpaper.mnist.models.real.SimpleDenseModel'>"
options["stages"]['sparsify']["model"]["cls"] = "<class 'cplxpaper.mnist.models.real.SimpleDenseModelARD'>"
options["stages"]['fine-tune']["model"]["cls"] = "<class 'cplxpaper.mnist.models.real.SimpleDenseModelMasked'>"

In [ ]:
options["stages"]['fine-tune']['reset'] = True

In [ ]:
# options

In [ ]:
snapshots = auto.run(options, './test', "fashion-mnist-simple")

In [ ]:
from cplxpaper.auto.utils import load_snapshot

def get_terms_from_snapshots(*snapshots):
    history = {"early_history": []}
    for snapshot in snapshots:
        state = load_snapshot(snapshot)

        # extract train loss history
        for k, v in state["history"].items():
            history.setdefault(k, []).append(v)

        # extract history
        history["early_history"].append(state["early_history"])

    return {k: np.concatenate(v, axis=0) for k, v in history.items()}

In [ ]:
history = get_terms_from_snapshots(*snapshots)

fig, ax = plt.subplots(1, 1, figsize=(16, 5))
for name, values in history.items():
    if name == "early_history": continue
    ax.semilogy(values, label=name, alpha=0.6)

In [ ]:
load_snapshot(snapshots[0])["performance"]

In [ ]:
load_snapshot(snapshots[-1])["performance"]

In [ ]:
assert False

<br>

In [ ]:
cold = load_snapshot('./test/2-fine-tune cplx-mnist.gz')

In [ ]:

reim = zip(cold['model']["conv1.weight.real"].numpy(), cold['model']["conv1.weight.imag"].numpy())
for re, im in reim:
    plt.imshow((re*re+im*im)[0])
    plt.show()

<br>